First - two things need to happen with you email.
1. Allow less secure access to gmail (just search "less secure" in settings
2. Go to forwarding in Gmail settings and enable IMAP

In [1]:
import smtplib
import imaplib
import time
import email
import pandas as pd
import datetime
import os

# need these to send emails
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

In [2]:
login_email  = "your email"
login_password = "your password"
imap_ssl = "" # e.g. imap.gmail.com
smtp_ssl = "" # e.g. smtp.gmail.com

In [3]:
# login into email server and select inbox
mail = imaplib.IMAP4_SSL(imap_ssl)
mail.login(login_email,login_password)
mail.select('inbox')

# get list of email ids
typ, data = mail.search(None, 'ALL')
mail_ids = data[0]
id_list = mail_ids.split()

In [4]:
def get_email_type(email_subject):
    if 'Sign-Up' in email_subject:
        return('sign_up')
    elif 'Cancellation' in email_subject:
        return('cancel')
    else:
        return('other')

In [5]:
def get_name_email_from_SONA(body_text, email_type):
    start = body_text.find('The participant ') + len('The participant ')
    
    if email_type == 'sign_up':
        end = body_text.find(' signed up for the study')
    elif email_type == 'cancel':
        end = body_text.find(' cancelled his/her sign-up for the study')
    else:
        raise Exception('Function only works for sign-ups and cancellations')
    
    return(body_text[start:end].split(' '))

In [6]:
def get_timeslot_from_SONA(body_text):
    start = body_text.find('to take place on ') + len('to take place on ')
    end = body_text.find(' in the location')
    
    month_dict = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 
              'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
    
    timeslot = body_text[start:end].split(' ')
    
    date = datetime.date(year = int(timeslot[3]), month = month_dict[timeslot[1]], day = int(timeslot[2].strip(',')))
    start_time = datetime.datetime.strptime(timeslot[4] + timeslot[5], '%I:%M%p').time()
    end_time = datetime.datetime.strptime(timeslot[-2] + timeslot[-1], '%I:%M%p').time()
    
    return(date, start_time, end_time)

In [7]:
# what text is in the subject that will indicate it is the study you are looking for
study_indicator = 'Part 2 of 2 Strategy Game'

cols = {'email_type':[], 'first_name':[], 'last_name':[], 'email':[],
       'date':[], 'start_time':[], 'end_time':[]}

# go through the emails
for ids in id_list:
    typ, data = mail.fetch(ids, '(RFC822)')

    # go through the parts of the email
    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            if study_indicator in msg['subject']:
                cols['email_type'].append(get_email_type(msg['subject']))
                
                name_email = get_name_email_from_SONA(msg.get_payload(decode=True).decode('utf-8'), cols['email_type'][-1])
                cols['first_name'].append(name_email[0])
                cols['last_name'].append(name_email[1])
                cols['email'].append(name_email[2].strip('<>'))
                
                date, start_time, end_time = get_timeslot_from_SONA(msg.get_payload(decode=True).decode('utf-8'))
                cols['date'].append(date)
                cols['start_time'].append(start_time)
                cols['end_time'].append(end_time)

In [8]:
df = pd.DataFrame(cols)[['email_type', 'date', 'start_time', 'end_time', 'first_name', 'last_name', 'email']]

I've successfully scraped my gmail to get the emails of those that have signed up for my study. These are all emails that SONA sends automatically.

Now the question is, can I take these names/emails and insert them into a formatted email.

embedding the images and new data got a bit tricky.  
- I had to add "cid:" prior to all of the image source indicators in the html file.
- I had to change all of the single braces other than those enclosing name, datem, and time to double braces

In [38]:
def send_reminder_email(login_email, login_password, from_display_name, to_email, subject, email_template, name, time, date):
    # Create message container - the correct MIME type is multipart/alternative.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = from_display_name
    msg['To'] = to_email
    
    with open('email_template.html', 'r') as f:
        email_template = f.read()
    html = email_template.format(name = name, time = time, date = date)
    
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    
    # attach the immages
    for i in [i for i in os.listdir('./images') if '.png' in i]:
        fp = open('./images/' + i, 'rb')
        msg_image = MIMEImage(fp.read())
        fp.close()

        msg_image.add_header('Content-ID', '<images/' + i + '>')
        msg.attach(msg_image)

    # set up the SMTP server
    s = smtplib.SMTP_SSL(smtp_ssl)
    s.login(login_email, login_password)

    # sendmail function takes 3 arguments: sender's address, recipient's address
    # and message to send - here it is sent as one string.
    s.sendmail(from_email, to_email, msg.as_string())
    s.quit()
    print('email sent to ' + to_email)

In [39]:
send_reminder_email(
login_email = login_email,
login_password = login_password,
from_display_name = 'Duke Decision Research',
to_email = "alexdsbreslav@gmail.com",
subject = "Personal Study Reminder: Part 2 of 2 Strategy Game for Food and Money",
email_template = email_template,
name = 'Alex',
time = df.start_time[1].strftime("%I:%M %p"),
date = df.date[1].strftime('%A, %B %d'))

email sent to alexdsbreslav@gmail.com
